In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
print(sys.path)
import pandas as pd
from src.robo import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *

tail = regression_times + 50
use_cols = ['open_time', 'close']

estimator = 'lightgbm'

In [ ]:
symbol = 'BTCUSDT'
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
df = get_data(symbol, save_database=False, interval='1h', tail=tail, adjust_index=True)
df.to_csv('log_after_1st_get_data.csv', sep=';', index=False)
df

In [ ]:
df = calc_RSI(df)
df.to_csv('log_after_1st_calc_RSI.csv', sep=';', index=False)
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
numeric_features = ['close', 'rsi']
df, _ = regresstion_times(df, numeric_features, regression_times)
df.to_csv('log_after_1st_regresstion_times.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
window = 14
close_price = 'close'
count = df.shape[0]
df.iloc[count-1:count]['rsi'] = 0


#print(df[['close', 'rsi']].tail())
# _df = df.tail(window + 10)[[close_price, 'rsi']].copy()
_df = df[[close_price, 'rsi']].tail(window+50).copy()
_count = _df.shape[0]
#print(_df)
_df['change'] = _df[close_price].diff()
_df['gain'] = _df.change.mask(_df.change < 0, 0.0)
_df['loss'] = -_df.change.mask(_df.change > 0, -0.0)
_df['avg_gain'] = rma(_df.gain.to_numpy(), window)
_df['avg_loss'] = rma(_df.loss.to_numpy(), window)

_df['rs'] = _df.avg_gain / _df.avg_loss
_df['rsi'] = 100 - (100 / (1 + _df.rs))
#print(_df)

#print(df.iloc[count-1:count]['rsi'])
#print(_df.iloc[_count-1:_count]['rsi'])

df.update(_df.iloc[_count-1:_count]['rsi'])

df.tail()

In [ ]:
cont = 0

In [ ]:
#while True:

# df = get_data(symbol, save_database=False, interval='1h', tail=tail)
_df = get_klines(symbol, interval='1h', max_date=None, limit=1, adjust_index=True)
_df.to_csv(f'log_after_{cont}_get_klines.csv', sep=';', index=False)

#df = pd.concat([df, _df]).drop_duplicates(keep='last')
if _df.index.isin(df.index):
    df.update(_df)
else:
    df = pd.concat([df, _df])   
df.to_csv(f'log_after_{cont}_merge.csv', sep=';', index=False)

df = calc_RSI(df, last_one=True)
df.to_csv(f'log_after_{cont}_calc_RSI.csv', sep=';', index=False)

df, _ = regresstion_times(df, numeric_features, regression_times, last_one=True)
df.to_csv(f'log_after_{cont}_regresstion_times.csv', sep=';', index=False)

df[['open_time'] + numeric_features].to_csv('log_experiment_data.log', sep=';', index=False)
df_predict = experiment.predict_model(model, df.tail(1))
df_predict.to_csv(f'log_after_{cont}_df_predict.csv', sep=';', index=False)

operacao = df_predict['prediction_label'].values[0]
if (operacao.startswith('SOBE') or operacao.startswith('CAI')):
    send_message(df_predict)
time.sleep(sleep_refresh)
cont += 1

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'open_time': [1, 55, 32, 234], 'close': [3, 4, 5, 6], 'C': [5, 6, 34, 5], 'D': [7, 8, 54, 66], 'label': ['A', 'A', 'B', 'B']}, index=[0,1,2,3])
df1

In [ ]:
df2 = pd.DataFrame({'open_time': [55], 'close': [77]}, index=[1])
df2

In [ ]:
i = df2.iloc[df2.shape[0]-1:df2.shape[0]].index
i

In [ ]:
df2.index.isin(df1.index)

In [ ]:
_df = get_klines('BTCUSDT', interval='1h', max_date=None, limit=200, adjust_index=True)
_df

In [ ]:
from pycaret.classification.oop import ClassificationExperiment

ca = ClassificationExperiment()
setup = ca.setup(_df, target='symbol')

In [ ]:
best = setup.compare_models()

best

In [ ]:
setup.pull().index[0]

In [2]:
import datetime 
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

'2023-09-08 10:54:22'

In [9]:
import pandas as pd
import datetime
import os

df_resultado_simulacao = pd.DataFrame()
if (os.path.exists('resultado_simulacao.csv')):
    df_resultado_simulacao = pd.read_csv('resultado_simulacao.csv', sep=';')

result_simulado = {}
result_simulado['model_name'] = 'BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_1'
result_simulado['data'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
result_simulado['symbol'] = 'BTCUSDT'
result_simulado['estimator'] = 'lightgbm'
result_simulado['train_size'] = 0.7 
result_simulado['start_train_date'] = '2023-06-01'
result_simulado['start_test_date'] = '2023-08-08'
result_simulado['regression_times'] = 24*30
result_simulado['regression_profit_and_loss'] = 24
result_simulado['stop_loss'] = 2.0
result_simulado['fold'] = 3
result_simulado['saldo_inicial'] = 100.00
result_simulado['saldo_final'] = 91.87

print(result_simulado)

df = pd.DataFrame.from_dict([result_simulado])

df_resultado_simulacao = pd.concat([df_resultado_simulacao, df], ignore_index=True)
df_resultado_simulacao.sort_values('saldo_final', inplace=True)

df_resultado_simulacao.to_csv('resultado_simulacao.csv', sep=';', index=False)

{'model_name': 'BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_1', 'data': '2023-09-08 11:01:40', 'symbol': 'BTCUSDT', 'estimator': 'lightgbm', 'train_size': 0.7, 'start_train_date': '2023-06-01', 'start_test_date': '2023-08-08', 'regression_times': 720, 'regression_profit_and_loss': 24, 'stop_loss': 2.0, 'fold': 3, 'saldo_inicial': 100.0, 'saldo_final': 91.87}


In [2]:
from binance.client import Client
client = Client()
klines = client.get_historical_klines(symbol='BTCUSDT', interval='1h', start_str='2010-01-01', limit=1000)



In [7]:
import pandas as pd


all_klines_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                   'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'number_of_trades', 'ignore', 'close_time']

df_klines = pd.DataFrame(data=klines, columns=all_klines_cols)[columns]
df_klines['open_time'] = pd.to_datetime(df_klines['open_time'], unit='ms')
df_klines['close_time'] = pd.to_datetime(df_klines['close_time'], unit='ms')

In [8]:
df_klines

,open_time,open,high,low,close,volume,quote_asset_volume,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,number_of_trades,ignore,close_time
0,2017-08-17 04:00:00,4261.48000000,4313.62000000,4261.32000000,4308.83000000,47.18100900,202366.13839304,35.16050300,150952.47794304,171,0,2017-08-17 04:59:59.999
1,2017-08-17 05:00:00,4308.83000000,4328.69000000,4291.37000000,4315.32000000,23.23491600,100304.82356749,21.44807100,92608.27972836,102,0,2017-08-17 05:59:59.999
2,2017-08-17 06:00:00,4330.29000000,4345.45000000,4309.37000000,4324.35000000,7.22969100,31282.31266989,4.80286100,20795.31722405,36,0,2017-08-17 06:59:59.999
3,2017-08-17 07:00:00,4316.62000000,4349.99000000,4287.41000000,4349.99000000,4.44324900,19241.05829986,2.60229200,11291.34701534,25,0,2017-08-17 07:59:59.999
4,2017-08-17 08:00:00,4333.32000000,4377.85000000,4333.32000000,4360.69000000,0.97280700,4239.50358563,0.81465500,3552.74681715,28,0,2017-08-17 08:59:59.999
...,...,...,...,...,...,...,...,...,...,...,...,...
52995,2023-09-08 15:00:00,25831.24000000,25900.00000000,25818.40000000,25844.64000000,1676.76165000,43366622.55950750,1014.86634000,26248043.92137060,26793,0,2023-09-08 15:59:59.999
52996,2023-09-08 16:00:00,25844.64000000,25875.15000000,25719.80000000,25796.69000000,1772.39260000,45739438.39591560,896.55165000,23141016.37179960,36637,0,2023-09-08 16:59:59.999
52997,2023-09-08 17:00:00,25796.69000000,25849.10000000,25746.00000000,25829.01000000,868.12163000,22401500.02390530,463.73080000,11965796.64501530,24745,0,2023-09-08 17:59:59.999
52998,2023-09-08 18:00:00,25829.01000000,25834.97000000,25766.21000000,25798.34000000,710.58889000,18336134.65578370,321.07817000,8284515.57731030,24054,0,2023-09-08 18:59:59.999
